In [1]:
import pandas as pd
import openpyxl as op

### Задание 7

In [ ]:
dfe = pd.read_excel('../data/документация/КТ_Кировский.xlsx')
dfc = pd.read_csv('../data/документация/выгрузки/sp.csv')

dfec = dfe.copy(deep=True)
dfcc = dfc.copy(deep=True)

dfcc['Сумма'] = dfcc['Сумма'].replace(',', '.', regex=True).astype(float)
dfcc['Пени'] = dfcc['Пени'].replace(',', '.', regex=True).astype(float)
dfcc['ГосПошлина'] = dfcc['ГосПошлина'].replace(',', '.', regex=True).astype(float)

In [ ]:
df = dfcc.copy(deep=True)
df['Номер ИД'] = df['Номер ИД'].str.split(r',\s*')
df = df.explode('Номер ИД')

df[df['Номер ИД'] == "2-88-1941/2022"], df[df['Номер ИД'] == "2-88-1942/2022"]

In [ ]:
merged_df = pd.merge(dfec, dfcc[['Номер ИД', 'Сумма', 'Пени', 'ГосПошлина']], how='left').drop_duplicates()
merged_df['Общий долг'] = merged_df[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

print(f"Null - {merged_df[merged_df['Общий долг'].isna()].shape}, All - {merged_df.shape}")

Null - (0, 35), All - (2520, 35)


In [ ]:
pd.set_option('display.max_columns', None)
merged_df[merged_df['Общий долг'] == 0]

In [108]:
merged_df.to_excel('../data/документация/КТ_Кировский_ОбщийДолг.xlsx')

In [72]:
dfcc.dtypes

Номер ИД       object
Номер           int64
Счет          float64
Сумма         float64
Пени          float64
ГосПошлина    float64
dtype: object

### Задание 8 - Общий отчет

In [ ]:
ls = pd.read_csv("../data/документация/выгрузки/ls.csv")
sv = pd.read_excel("../data/документация/выгрузки/Стадии взысканий.xlsx")

lscopy = ls.copy(deep=True)
lscopy = pd.merge(lscopy, sv, how="left")

lscopy['Сумма'] = pd.to_numeric(lscopy['Сумма'].replace(',', '.', regex=True).astype(float))
lscopy['ROW_ID_Взыскания'] = pd.to_numeric(lscopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
lscopy['НомерДела'] = pd.to_numeric(lscopy['НомерДела'], errors='coerce').fillna(0).astype(int)
lscopy['Месяц долга'] = pd.to_datetime(lscopy['Месяц долга'])

Payment = pd.read_csv("../data/документация/выгрузки/np.csv")

# closedBills = pd.read_csv("../data/документация/выгрузки/zs.csv")
# closedBills["ДатаЗакрытия"] = pd.to_datetime(closedBills["ДатаЗакрытия"])

filter1 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Абонент Абонент Абонент.xlsx")
filter2 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Лица Призванные Наследовать.xlsx")
filter3 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Наниматель Жилого Помещения.xlsx")
filter4 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Юридические Лица.xlsx")
filter5 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Нежилое Помещение.xlsx")
total_filter = pd.concat([filter1, filter2, filter3, filter4, filter5])

df_ost = pd.read_csv("../data/документация/выгрузки/Остальное.csv")
df_ces = pd.read_csv("../data/документация/выгрузки/.csv")
df_mun = pd.read_csv("../data/документация/выгрузки/Мунжилье.csv")

def CategoryDZ(df):
    
    date = pd.to_datetime('2025-02-01')

    df['months_diff'] = (
        (date.year - df['Месяц долга'].dt.year) * 12
        + (date.month - df['Месяц долга'].dt.month)
    )

    df['years_diff'] = df['months_diff'] // 12

    def create_row(aLS, aNS, sum):
        return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

    dfc = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

    # Отчетный месяц
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff == 0')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff == 0')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff == 0')['Сумма'].sum())])

    # от 2 до 3 месяцев после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('3 > months_diff >= 2')['Номер ЛЦ'].nunique(),
                                    df.query('3 > months_diff >= 2')['ROW_ID_Взыскания'].nunique(),
                                    df.query('3 > months_diff >= 2')['Сумма'].sum())])

    # от 3 месяцев до 3 лет после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff >= 3 and years_diff <= 3')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['Сумма'].sum())])

    # свыше 3 лет
    dfc = pd.concat([dfc, create_row(
                                    df.query('years_diff >= 3')['Номер ЛЦ'].nunique(),
                                    df.query('years_diff >= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('years_diff >= 3')['Сумма'].sum())])

    # не было оплат
    noPayment = df[~df['Счет-Наниматель'].isin(Payment)]

    dfc = pd.concat([dfc, create_row(
                                    noPayment['Номер ЛЦ'].nunique(),
                                    noPayment['ROW_ID_Взыскания'].nunique(),
                                    noPayment['Сумма'].sum())])
    
    # Свыше 3-х лет+все остальные периоды по этим (закрытым) л/с
    # closedBills['yearsdiff'] = (date.year - closedBills["ДатаЗакрытия"].dt.year) + \
    #                      ((date.month - closedBills["ДатаЗакрытия"].dt.month) / 12)
    # periodFilter = closedBills[closedBills["yearsdiff"] >= 3]

    allPeriods = df[df["Значение"] == 2]

    dfc = pd.concat([dfc, create_row(
                                    allPeriods['Номер ЛЦ'].nunique(),
                                    allPeriods['ROW_ID_Взыскания'].nunique(),
                                    allPeriods['Сумма'].sum())])
    
    # ДЗ менее 3000 р 
    small_sum = df.groupby(['Номер ЛЦ', 'Счет-Наниматель', 'ROW_ID_Взыскания'])['Сумма'].sum().reset_index()
    small_sum = small_sum[small_sum['Сумма'] < 3000]
    
    small_sum_filter = df[df['Номер ЛЦ'].isin(small_sum['Номер ЛЦ'])]
    
    dfc = pd.concat([dfc, create_row(
                                    len(small_sum),  
                                    small_sum_filter['ROW_ID_Взыскания'].nunique(),            
                                    small_sum['Сумма'].sum())]) 
    
    # Без ФИО, Лица призванные наследовать, Юридические лица, Нежилое помещение, Наниматель жилого помещения, Несовершеннолетние
    dfcopy_filtered = df[df["ФИО"].isin(total_filter["ФИО"])]

    dfc = pd.concat([dfc, create_row(
                                    dfcopy_filtered['Номер ЛЦ'].nunique(),
                                    dfcopy_filtered['ROW_ID_Взыскания'].nunique(),
                                    dfcopy_filtered['Сумма'].sum())])
    return dfc


def Category(df, start_row):
    dfcopy = df.copy(deep=True)
    dfcopy = pd.merge(dfcopy, sv, how="left")

    def updateExcel(df, start_row):
        wb = op.load_workbook('./отчеты/Отчет ежемесячный КT2(копия).xlsx')
        wba = wb.active
        start_col = 6 

        for row_index, (index, row) in enumerate(df.iterrows(), start=start_row):  
            for col_index, (col_name, cell_value) in enumerate(row.items(), start=start_col):
                cell = wba.cell(row=row_index, column=col_index)
                cell.value = cell_value

        wb.save('./отчеты/Отчет ежемесячный КT2(копия).xlsx')

    dfcopy['Сумма'] = pd.to_numeric(dfcopy['Сумма'].replace(',', '.', regex=True).astype(float))
    dfcopy['ROW_ID_Взыскания'] = pd.to_numeric(dfcopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
    dfcopy['НомерДела'] = pd.to_numeric(dfcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
    dfcopy['Месяц долга'] = pd.to_datetime(dfcopy['Месяц долга'])

    # Досудебная
    Dosud = dfcopy[(dfcopy["Общая стадия"] == "Досубедная от месяца") | dfcopy["Фаза"].isna() | dfcopy["Общая стадия"].isna()]

    # Судебная
    Sud = dfcopy[dfcopy["Общая стадия"] == "Суд"]

    # ФССП
    Fssp = dfcopy[(dfcopy["Общая стадия"] == "ФССП")]

    total_df = pd.concat([CategoryDZ(Dosud), CategoryDZ(Sud), CategoryDZ(Fssp)], axis=1)

    updateExcel(total_df, start_row)


Category(df_ost, 17)
Category(df_ces, 27)
Category(df_mun, 37)



C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_76860\2571573937.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_76860\2571573937.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12


KeyError: 'Номер ЛЦ'

In [54]:
df_ost.shape[0], df_ces.shape[0], df_mun.shape[0]

(819331, 167269, 141475)

In [45]:
lscopy[lscopy['Номер ЛЦ'] == 265544]

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела,Значение,Общая стадия,Сомнительность
201381,1616.58,265544,1422766.0,2017-03-01,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Абонент Абонент Абонент,NaN,NaN,0,NaN,NaN,NaN,NaN,0,2.0,NaN,NaN
399935,1689.12,265544,1422766.0,2016-11-01,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Абонент Абонент Абонент,NaN,NaN,0,NaN,NaN,NaN,NaN,0,2.0,NaN,NaN
499827,1239.01,265544,1422766.0,2017-04-01,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Абонент Абонент Абонент,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [31]:
closedBills

,ROW_ID,Счета,Счета_ADD,Счет-Линейный участок,Улица-Лицевой счет,Номер,Подразделение,Тип,Код строения,Счет-Средство,...,ОКТМО,ГИС_ЕЛС,ГруппаОбъектов,ОбластьДоставки,Счет-Доставка,ЮЛ_Блокировка фонда,ЮЛ_Месяц блокировки,OLD_ROW_ID,ПримечаниеЮрист,yearsdiff
0,1025155,117175,1,-1,-1,123836,0,5,0,-1,...,NaN,60МН159882,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.333333
1,1082285,175037,1,-1,-1,149049,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
2,1082287,157793,1,-1,-1,149050,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
3,1082289,16682,1,-1,-1,149051,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
4,1082291,467064,1,-1,-1,149052,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4528,2047372,177988,1,-1,-1,4720952801,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
4529,3589095,3729883,1,-1,-1,402762,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
4530,3589096,3729885,1,-1,-1,402763,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667
4531,3709913,168621,1,-1,-1,411091,0,5,0,-1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,6.166667


In [33]:
date = pd.to_datetime('2025-02-01')

closedBills['yearsdiff'] = (date.year - closedBills["ДатаЗакрытия"].dt.year) + \
                         ((date.month - closedBills["ДатаЗакрытия"].dt.month) / 12)
periodFilter = closedBills[closedBills["yearsdiff"] >= 3]

allPeriods = df_ost[df_ost["Счет-Наниматель"].isin(periodFilter["ROW_ID"])]

In [34]:
allPeriods

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела


In [38]:
fzcopy[fzcopy["Счет-Наниматель"].isin(closedBills["Счет-Наниматель"])]

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела,Общая стадия,Сомнительность


In [19]:
closedBills[["Счет-Наниматель", "yearsdiff"]]

,Счет-Наниматель,yearsdiff
0,1824158,6.333333
1,1951860,6.166667
2,1951861,6.166667
3,1951872,6.166667
4,1951874,6.166667
...,...,...
4528,3934623,6.166667
4529,6332401,6.166667
4530,6332403,6.166667
4531,6453390,6.166667


In [34]:
date = pd.to_datetime('2025-02-01')
closedBills["ДатаЗакрытия"] = pd.to_datetime(closedBills["ДатаЗакрытия"])
closedBills['yearsdiff'] = date - closedBills["ДатаЗакрытия"]
periodFilter = closedBills[closedBills["yearsdiff"].astype(int) >= 3]
periodFilter['ДатаЗакрытия']

0      2018-10-31
1      2018-12-31
2      2018-12-31
3      2018-12-31
4      2018-12-31
          ...    
4528   2018-12-31
4529   2018-12-31
4530   2018-12-31
4531   2018-12-31
4532   2018-12-31
Name: ДатаЗакрытия, Length: 4533, dtype: datetime64[ns]

In [5]:
df_ost[df_ost['ROW_ID_Взыскания'].isna()]

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела
8,"12,31",1075,671881,2024-10-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Феклистова Лидия Ивановна,1950-10-15 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,"2355,81",1129,671966,2024-09-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Селезнева Валентина Александровна,1980-04-23 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,"4335,21",1129,671966,2024-12-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Селезнева Валентина Александровна,1980-04-23 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,"262,14",1137,671980,2024-11-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Бундин Василий Анатольевич,1979-10-05 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,"3035,02",1137,671980,2024-12-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Бундин Василий Анатольевич,1979-10-05 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819285,"3123,83",4720953428,2066902,2025-01-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Синдрякова Ольга Александровна,1962-09-01 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
819297,"1311,54",2008096407,1203191,2025-01-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Захарова Ольга Ивановна,1962-07-22 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
819303,"450,08",2008097407,1203211,2024-09-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Курганская Галина Викторовна,1971-10-30 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
819304,"1299,67",2008098707,1203239,2024-12-01 00:00:00,"ООО ""Концессии теплоснабжения"" Прямой договор",5625,Воронин Александр Анатольевич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Category(df_ost, 17)

In [ ]:
Category(df_ces, 27)

In [ ]:
Category(df_mun, 37)

In [80]:
df_ost.shape[0] + df_ces.shape[0] + df_mun.shape[0]

987369

In [81]:
fzcopy[fzcopy["ФИО"].isin(total_filter["ФИО"])].shape[0]

71982

In [5]:
fzcopy.dtypes

Сумма                             float64
Номер ЛЦ                            int64
Счет-Наниматель                   float64
Месяц долга                datetime64[ns]
Наименование                       object
ROW_ID_Организации                  int64
ФИО                                object
Дата рождения                      object
Дата смерти                        object
ROW_ID_Взыскания                    int64
Фаза                               object
Дата постановки на фазу            object
ДатНачДолга                        object
ДатКнцДолга                        object
НомерДела                           int64
Общая стадия                       object
Сомнительность                     object
dtype: object

In [ ]:
ls = pd.read_csv("../data/документация/выгрузки/ls.csv")
sv = pd.read_excel("../data/документация/выгрузки/Стадии взысканий.xlsx")

lscopy = ls.copy(deep=True)
lscopy = pd.merge(lscopy, sv, how="left")

lscopy['Сумма'] = pd.to_numeric(lscopy['Сумма'].replace(',', '.', regex=True).astype(float))
lscopy['ROW_ID_Взыскания'] = pd.to_numeric(lscopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
lscopy['НомерДела'] = pd.to_numeric(lscopy['НомерДела'], errors='coerce').fillna(0).astype(int)
lscopy['Месяц долга'] = pd.to_datetime(lscopy['Месяц долга'])

Payment = pd.read_csv("../data/документация/выгрузки/np.csv")

# closedBills = pd.read_csv("../data/документация/выгрузки/zs.csv")
# closedBills["ДатаЗакрытия"] = pd.to_datetime(closedBills["ДатаЗакрытия"])

filter1 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Абонент Абонент Абонент.xlsx")
filter2 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Лица Призванные Наследовать.xlsx")
filter3 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Наниматель Жилого Помещения.xlsx")
filter4 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Юридические Лица.xlsx")
filter5 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Нежилое Помещение.xlsx")
total_filter = pd.concat([filter1, filter2, filter3, filter4, filter5])

df_ost = pd.read_csv("../data/документация/выгрузки/Остальное.csv")
df_ces = pd.read_csv("../data/документация/выгрузки/Цессии.csv")
df_mun = pd.read_csv("../data/документация/выгрузки/Мунжилье.csv")

def CategoryDZ(df):
    
    date = pd.to_datetime('2025-02-01')

    df['months_diff'] = (
        (date.year - df['Месяц долга'].dt.year) * 12
        + (date.month - df['Месяц долга'].dt.month)
    )

    df['years_diff'] = df['months_diff'] // 12

    def create_row(aLS, aNS, sum):
        return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

    dfc = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

    # Отчетный месяц
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff == 0')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff == 0')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff == 0')['Сумма'].sum())])

    # от 2 до 3 месяцев после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('3 > months_diff >= 2')['Номер ЛЦ'].nunique(),
                                    df.query('3 > months_diff >= 2')['ROW_ID_Взыскания'].nunique(),
                                    df.query('3 > months_diff >= 2')['Сумма'].sum())])

    # от 3 месяцев до 3 лет после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff >= 3 and years_diff <= 3')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['Сумма'].sum())])

    # свыше 3 лет
    dfc = pd.concat([dfc, create_row(
                                    df.query('years_diff >= 3')['Номер ЛЦ'].nunique(),
                                    df.query('years_diff >= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('years_diff >= 3')['Сумма'].sum())])

    # не было оплат
    noPayment = df[~df['Счет-Наниматель'].isin(Payment)]

    dfc = pd.concat([dfc, create_row(
                                    noPayment['Номер ЛЦ'].nunique(),
                                    noPayment['ROW_ID_Взыскания'].nunique(),
                                    noPayment['Сумма'].sum())])
    
    # Свыше 3-х лет+все остальные периоды по этим (закрытым) л/с
    # closedBills['yearsdiff'] = (date.year - closedBills["ДатаЗакрытия"].dt.year) + \
    #                      ((date.month - closedBills["ДатаЗакрытия"].dt.month) / 12)
    # periodFilter = closedBills[closedBills["yearsdiff"] >= 3]

    allPeriods = df[df["Значение"] == 2]

    dfc = pd.concat([dfc, create_row(
                                    allPeriods['Номер ЛЦ'].nunique(),
                                    allPeriods['ROW_ID_Взыскания'].nunique(),
                                    allPeriods['Сумма'].sum())])
    
    # ДЗ менее 3000 р 
    small_sum = df.groupby(['Номер ЛЦ', 'Счет-Наниматель', 'ROW_ID_Взыскания'])['Сумма'].sum().reset_index()
    small_sum = small_sum[small_sum['Сумма'] < 3000]
    
    small_sum_filter = df[df['Номер ЛЦ'].isin(small_sum['Номер ЛЦ'])]
    
    dfc = pd.concat([dfc, create_row(
                                    len(small_sum),  
                                    small_sum_filter['ROW_ID_Взыскания'].nunique(),            
                                    small_sum['Сумма'].sum())]) 
    
    # Без ФИО, Лица призванные наследовать, Юридические лица, Нежилое помещение, Наниматель жилого помещения, Несовершеннолетние
    dfcopy_filtered = df[df["ФИО"].isin(total_filter["ФИО"])]

    dfc = pd.concat([dfc, create_row(
                                    dfcopy_filtered['Номер ЛЦ'].nunique(),
                                    dfcopy_filtered['ROW_ID_Взыскания'].nunique(),
                                    dfcopy_filtered['Сумма'].sum())])
    return dfc


def Category(df, start_row):
    dfcopy = df.copy(deep=True)
    dfcopy = pd.merge(dfcopy, sv, how="left")

    def updateExcel(df, start_row):
        wb = op.load_workbook('./отчеты/Отчет ежемесячный КT2(копия).xlsx')
        wba = wb.active
        start_col = 6 

        for row_index, (index, row) in enumerate(df.iterrows(), start=start_row):  
            for col_index, (col_name, cell_value) in enumerate(row.items(), start=start_col):
                cell = wba.cell(row=row_index, column=col_index)
                cell.value = cell_value

        wb.save('./отчеты/Отчет ежемесячный КT2(копия).xlsx')

    dfcopy['Сумма'] = pd.to_numeric(dfcopy['Сумма'].replace(',', '.', regex=True).astype(float))
    dfcopy['ROW_ID_Взыскания'] = pd.to_numeric(dfcopy['ROW_ID_Взыскание'], errors='coerce').fillna(0).astype(int)
    dfcopy['НомерДела'] = pd.to_numeric(dfcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
    dfcopy['Месяц долга'] = pd.to_datetime(dfcopy['Месяц долга'])

    # Досудебная
    Dosud = dfcopy[(dfcopy["Общая стадия"] == "Досубедная от месяца") | dfcopy["Фаза"].isna() | dfcopy["Общая стадия"].isna()]

    # Судебная
    Sud = dfcopy[dfcopy["Общая стадия"] == "Суд"]

    # ФССП
    Fssp = dfcopy[(dfcopy["Общая стадия"] == "ФССП")]

    total_df = pd.concat([CategoryDZ(Dosud), CategoryDZ(Sud), CategoryDZ(Fssp)], axis=1)

    updateExcel(total_df, start_row)


Category(df_ost, 17)
Category(df_ces, 27)
Category(df_mun, 37)

In [56]:
df_mun

,Unnamed: 0,Сумма,Номер ЛС,Счет,Месяц долга,Наименование,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела,НебылоОплат,ЛицевойЗакрыт,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскание
0,0,1962.06,1949778407,1194745,2020-12-01,"ООО ""Концессии теплоснабжения"" Прямой договор",NaN,NaN,NaN,NaN,NaN,0,0,5625,Сироткин Александр Николаевич,1961-12-01,NaN,NaN
1,1,2532.47,442470,3269944,2019-05-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Исполнительное производство Сдача в ФССП,2024-01-30,2019-01-01,2023-07-01,2.148528e+09,0,0,5625,Лица призванные наследовать,NaN,NaN,1288004.0
2,2,3484.62,235635,1383503,2023-01-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Исполнительное производство Сдача в ФССП,2024-04-02,2020-01-01,2023-04-01,2.148548e+09,0,0,5625,Анишина Надежда Владимировна,1922-07-01,NaN,1308021.0
3,3,934.11,1265478507,865031,2024-03-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Закрыто,2025-02-25,2024-01-01,2024-03-01,2.148706e+09,0,0,5625,Лимасов Юрий Михайлович,1968-06-12,NaN,1468648.0
4,4,1100.96,1263859507,848958,2021-05-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Закрыто,2021-09-01,2021-05-01,2021-07-01,2.147960e+09,0,0,5625,Синицына Анна Вячеславовна,1983-08-12,NaN,706219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138213,138213,2731.72,1263210907,842524,2024-08-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Исполнительное производство Сдача в ФССП,2025-03-24,2019-10-01,2024-11-01,2.148838e+09,0,0,5625,Лепехина Юлия Александровна,1983-01-23,NaN,1609181.0
138214,138214,1634.64,135282,1061435,2020-11-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Закрыто,2021-10-01,2020-10-01,2021-04-01,2.147900e+09,0,0,5625,Наниматель жилого помещения,NaN,NaN,645497.0
138215,138215,662.49,315415,1563488,2022-11-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Заявление по судебному приказу Возврат,2024-04-25,2020-10-01,2023-04-01,2.148545e+09,0,0,5625,Городской округ Город-герой Волгоград,NaN,NaN,1305754.0
138216,138216,633.91,247139,1398024,2024-05-01,"ООО ""Концессии теплоснабжения"" Прямой договор",Заявление по судебному приказу Печать документов,2025-02-28,2023-08-01,2024-12-01,2.025022e+10,0,0,5625,Сергеева Татьяна Васильевна,1960-02-19,NaN,1625285.0
